In [77]:
import json

json_file = '/shared/hzhangal/Projects/zero-event-extraction/quizlet6/quizlet6_data/extracted_events.json'
file_name = 'quizlet6_data/L0C04AT6W.rsd.txt'

with open(json_file) as f:
    extracted_events = json.load(f)
    #print(extracted_events.keys())
    #print("\n")
    #print(extracted_events[file_name]['events'][1][0])
    original_content = extracted_events[file_name]['original_content']
    print(extracted_events[file_name]['events'][1][0])
    # Three keys for each file: original_content, sentences, events
    # "events" is a list
    # Things in "events":
    #     Each sent has a list
    #     In each list, there is a list of dict
    #     Each dict contains the info about an event
    #print(' '.join(extracted_events[file_name]['sentences']))

{'trigger': {'position': [1, 2], 'type': 'Life_Die_Unspecified'}, 'arguments': [{'position': [0, 1], 'role': 'Killer', 'entity_type': 'per'}], 'sentence': 'Hundreds Stricken by Unknown Illness After Apparently Dining at Ohio Chipotle: Report', 'tokens': ['Hundreds', 'Stricken', 'by', 'Unknown', 'Illness', 'After', 'Apparently', 'Dining', 'at', 'Ohio', 'Chipotle', ':', 'Report']}


In [ ]:
def sent_end_pos(sentences):
    end_pos = []
    count = 0
    for sent in sentences:
        count += len(sent)
        end_pos.append(count)
        count += 1
    return end_pos

sent_end_pos(extracted_events[file_name]['sentences'])

In [79]:
import pprint
pp = pprint.PrettyPrinter(indent=4)

def ee_to_ta(file_name):
    with open(json_file) as f:
        extracted_events = json.load(f)
        original_content = extracted_events[file_name]['original_content']
        sentences = extracted_events[file_name]['sentences']
        events = extracted_events[file_name]['events']
        text = ' '.join(sentences)
        
    tokens = []
    relations = []
    constituents = []
    sentence_id = -1
    for sent in events:
        sentence_id += 1
        if len(sent) > 0:
            tokens.extend(sent[0]['tokens'])
            for event in sent:
                constituents.append({"start": event['trigger']['position'][0], \
                                     "end": event['trigger']['position'][1], \
                                     "label": "event", \
                                     "properties": {'sentence_id': sentence_id, \
                                                    'predicate': ' '.join(sent[0]['tokens'][event['trigger']['position'][0]:event['trigger']['position'][1]]),\
                                                   'SenseNumber': '', \
                                                   }, \
                                     "score": 1.0, \
                                    })
    pp.pprint(constituents)
ee_to_ta('quizlet6_data/L0C04AT6W.rsd.txt')

[   {   'end': 2,
        'label': 'event',
        'properties': {   'SenseNumber': '',
                          'predicate': 'Stricken',
                          'sentence_id': 1},
        'score': 1.0,
        'start': 1},
    {   'end': 8,
        'label': 'event',
        'properties': {   'SenseNumber': '',
                          'predicate': 'Dining',
                          'sentence_id': 1},
        'score': 1.0,
        'start': 7},
    {   'end': 1,
        'label': 'event',
        'properties': {   'SenseNumber': '',
                          'predicate': 'Hundreds',
                          'sentence_id': 1},
        'score': 1.0,
        'start': 0},
    {   'end': 13,
        'label': 'event',
        'properties': {   'SenseNumber': '',
                          'predicate': 'Report',
                          'sentence_id': 1},
        'score': 1.0,
        'start': 12},
    {   'end': 5,
        'label': 'event',
        'properties': {   'SenseNumber': '',
 

In [ ]:
def ee_to_ta(file_name):
    with open(json_file) as f:
        extracted_events = json.load(f)
        original_content = extracted_events[file_name]['original_content']
        sentences = extracted_events[file_name]['sentences']
        events = extracted_events[file_name]['events']
        text = ' '.join(sentences)
        
    tokens = []
    relations = []
    constituents = []
    sentence_id = -1
    for sent in events:
        sentence_id += 1
        if len(sent) > 0:
            tokens.extend(sent[0]['tokens'])
            for event in sent:
                constituents.append({"start": event['trigger']['position'][0], \
                                     "end": event['trigger']['position'][1], \
                                     "label": "event", \
                                     "properties": {'sentence_id': sentence_id, \
                                                    'predicate': ' '.join(sent[0]['tokens'][event['trigger']['position'][0]:event['trigger']['position'][1]]),\
                                                   'SenseNumber': '', \
                                                   }, \
                                     "score": 1.0, \
                                    })
            
    
    ta = {}
    ta["corpusID"] = file_name
    ta["id"] = ""
    ta["sentences"] = {"generator": "tsvx_to_ta", "score":1.0, "sentenceEndPositions": sent_end_pos(sentences)}
    ta["text"] = original_content
    ta["tokens"] = tokens
    ta["views"] = [{"viewData": [{"constituents": constituents, \
                                  "generator": "cogcomp_kairos_event_ie_v1.0", \
                                  "relations": relations, \
                                  "score": 1.0, \
                                  "viewName": "event_extraction", \
                                  "viewType": 'edu.illinois.cs.cogcomp.core.datastructures.textannotation.PredicateArgumentView', \
                                 }],\
                    'viewName': 'Event_extraction'
                   }]
    return ta

ee_to_ta('quizlet6_data/L0C04AT6W.rsd.txt')

In [ ]:
def ee_to_ta(dir_name, file_name):
    
    
    
    my_dict = tsvx_reader(dir_name, file_name)
    sentEndPos = []
    constituents = []
    tokens = []
    relations = []
    
    for sent_dict in my_dict['sentences']:
        sentEndPos.append(sent_dict['sent_end_char'])
        tokens.extend(sent_dict['tokens'])
        
    for key, event_dict in my_dict['event_dict'].items():
        constituents.append({"start": event_dict['token_id'], \
                             "end": event_dict['token_id'] + 1, \
                             "label": "event", \
                             "properties": {'sentence_id': event_dict['sent_id'], \
                                            'predicate': [event_dict['mention']], \
                                            'SenseNumber': '', \
                                           }, \
                             "score": 1.0, \
                            })
        
    #for key, rel in my_dict["relation_dict"].items():
    #    relations.append({'relationName': rel['relation'], \
    #                      'srcConstituent': key[0] + 1, \
    #                      'targetConstituent': key[1] + 1, \
    #                     })
        
    ta = {}
    ta["corpusID"] = my_dict['doc_id']
    ta["id"] = ""
    ta["sentences"] = {"generator": "tsvx_to_ta", "score":1.0, "sentenceEndPositions": sentEndPos}
    ta["text"] = my_dict['doc_content']
    ta["tokens"] = tokens
    ta["views"] = [{"viewData": [{"constituents": constituents, \
                                  "generator": "cogcomp_kairos_event_ie_v1.0", \
                                  "relations": relations, \
                                  "score": 1.0, \
                                  "viewName": "event_extraction", \
                                  "viewType": 'edu.illinois.cs.cogcomp.core.datastructures.textannotation.PredicateArgumentView', \
                                 }],\
                    'viewName': 'Event_extraction'
                   }]
    return ta